In [ ]:
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

import requests
from bs4 import BeautifulSoup
import os
from selenium import webdriver
import json
from pydap.client import open_url
from pydap.cas.urs import setup_session
from pathlib import Path  
import pandas as pd
import numpy as np

In [ ]:
def urls_papser(url):
    for str_  in url.split('/'):
        if str_.startswith('2'):
            item = str_
    return item


def get_data(url):

    headers = {
        "user-agent": "User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.160 YaBrowser/22.5.4.904 Yowser/2.5 Safari/537.36"
    }
    req = requests.get(url, headers)

    with open('nasa.html', 'w') as file:
        file.write(req.text)

    with open('nasa.html') as file:
        src  = file.read()
    soup = BeautifulSoup(src, 'lxml')
    tr_tag = soup.find_all('tr')

    dir_urls = []

    for tag in tr_tag:
        try:
            dir_url = 'https://oco2.gesdisc.eosdis.nasa.gov/opendap/OCO2_L2_Lite_FP.10r/' + tag.find('td').find('a').get('href')
            dir_urls.append(dir_url)
        except AttributeError:
            continue

    folder_name = None
    for url in dir_urls:
        item = urls_papser(url)
        folder_name = f"nasa_files/data_{item}"

        if os.path.exists(folder_name):
              print("The folder already exists!")
        else:
              os.mkdir(folder_name)

        browser=webdriver.Chrome('chromedriver',options=options)
        browser.get(url)

        soup_=BeautifulSoup(browser.page_source, 'lxml')

        url_ = None
        for scripts in soup_.select("script[type='application/ld+json']"):
            url_ = json.loads(scripts.get_text(strip=True))['dataset']

      

      for i in range(len(url_)):
        #print(url_[i]['sameAs'])

        url = url_[i]['sameAs'][:-5]
        session = setup_session(username=' YOUR USER NAME FROM EARTH DATA', password='YOUR PASSWORD EARTH DATA', check_url = url)
        dataset = open_url(url, session=session)

        requested_data_xco2 = dataset['xco2']
        requested_data_time = dataset['time']
        requested_data_latitude = dataset['latitude']
        requested_data_longitude = dataset['longitude']


        x_co2 = np.array(requested_data_xco2[:,:])
        time = np.array(requested_data_time[:,:])
        latitude = np.array(requested_data_latitude[:,:])
        longitude = np.array(requested_data_longitude[:,:])

        d = {'xco2': x_co2, 'time': time, 'latitude': latitude, 'longitude': longitude}
        df = pd.DataFrame(data=d)

        file_name = url_[i]['name']
        path = f"{folder_name}/{file_name}.csv"
        filepath = Path(path)  
        filepath.parent.mkdir(parents=True, exist_ok=True)  
        df.to_csv(filepath)

In [ ]:
url = 'https://oco2.gesdisc.eosdis.nasa.gov/opendap/OCO2_L2_Lite_FP.10r/contents.html'
get_data(url)